In [ ]:
# MIT License (https://opensource.org/licenses/MIT)
# Copyright 2018 Ryan Hausen

![morpheus](https://cdn.jsdelivr.net/gh/morpheus-project/morpheus/morpheus.svg)



This notebook can be run interactively in Google Colab
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/morpheus-project/morpheus/blob/master/examples/example_array.ipynb)

This is an example of using Morpheus to classify on a astronomical image.

In [ ]:
!pip install morpheus-astro

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from morpheus.classifier import Classifier
from morpheus.data import example

%matplotlib inline

### First get a sample image to examine.

This is a sample taken from GOODS.

In [ ]:
h, j, v, z = example.get_sample()

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
axes = np.array(axes).flatten()

for ax, arr, band in zip(axes, [h,j,v,z], 'HJVZ'):
    ax.set_title(f'{band} Image')
    ax.imshow(arr, origin='lower', cmap='gray')
    ax.grid(None)
plt.show()

### Classify image

Use [Classifer.classify_arrays](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier) to classify a nummpy array in memory. 
See documentation for classifying images from disk.

The output is a dictionary that contains a mapping for each pixel which representss the 
probability that a pixel belongs to one of the following classes:

- Spheroid
- Disk
- Irregular
- Point Source (Compact)
- Background

It also contains a mapping for `n`, which indicates how many times a classification was
recorded for that pixel

In [ ]:
morphs = Classifier.classify_arrays(h=h, j=j, v=v, z=z)

In [ ]:
f, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 15))
axes = np.array(axes).flatten()

for i, k in enumerate(morphs):
    axes[i].set_title(f'Output: {k}')
    axes[i].imshow(morphs[k], origin='lower', vmin=0, vmax=1, cmap='magma')
    axes[i].grid(None)
plt.show()

### Make a Segmap

Use [Classifier.make_segmap](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.make_segmap) 
to generate a segmentation map using the output and the H band flux.

In [ ]:
# the mask tells the segmentation mapping algorithm to ignore
# the areas unclassified as a result of the windowing classification
# method used.
mask = np.zeros_like(h, np.int)
mask[5:-5, 5:-5] = 1

segmap = Classifier.make_segmap(morphs, h, mask=mask)

plt.figure(figsize=(10, 10))
plt.imshow(segmap, origin='lower', cmap='gray')
plt.grid(None)
plt.show()

### Visualize the Morphological Classifications Using Color

Use [Classifier.colorize_rank_vote_output](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.colorize_rank_vote_output)  to make an RGB. The colors in the output have the following meanings:

Red = Spheroid
Blue = Disk
Green = Irregular
Yellow = Point Source (compact)
Back = Background

Note when Morpheus isn't doesn't output a strong classification for any one class they are colored white.

For more inforamtion on the coloring scheme see the [documentation](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.colorize_rank_vote_output).

In [ ]:
rgb = Classifier.colorize_rank_vote_output(morphs)
plt.imshow(rgb)
plt.grid(None)
plt.show()